In [1]:
#Loading libraries 
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, make_scorer

In [2]:
#loading data
train = pd.read_csv("/home/priya/Downloads/TitanicSinking/train.csv")
test = pd.read_csv("/home/priya/Downloads/TitanicSinking/test.csv")

In [3]:
target =  train['Survived']
del train['Survived']
totaldata = pd.concat([train, test],sort= False).reset_index(drop=True)

In [4]:
totaldata["FamilySize"]=totaldata["SibSp"]+totaldata["Parch"]+1
totaldata['Alone'] = totaldata['FamilySize'].map(lambda s : 1 if s == 1 else 0)
totaldata['Couple'] = totaldata['FamilySize'].map(lambda s : 1 if s==2 else 0)
totaldata['Family'] = totaldata['FamilySize'].map(lambda s : 1 if 3<=s else 0)

In [5]:
totaldata.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0,1,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0,1,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1,0,0


In [6]:
del totaldata['Parch']
del totaldata['SibSp']
del totaldata['Ticket']

In [7]:
totaldata.head()

,PassengerId,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family
0,1,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,NaN,S,2,0,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C85,C,2,0,1,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,NaN,S,1,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,C123,S,2,0,1,0
4,5,3,"Allen, Mr. William Henry",male,35.0,8.0500,NaN,S,1,1,0,0


In [8]:
totaldata.columns[totaldata.isnull().any()]

Index(['Age', 'Fare', 'Cabin', 'Embarked'], dtype='object')

In [9]:
totaldata['Title'] = totaldata['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
totaldata.head()

,PassengerId,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,NaN,S,2,0,1,0,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C85,C,2,0,1,0,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,NaN,S,1,1,0,0,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,C123,S,2,0,1,0,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,8.0500,NaN,S,1,1,0,0,Mr


In [10]:
del totaldata['Name']

In [11]:
totaldata.reset_index(inplace = True)
totaldata.drop('index',inplace=True,axis=1)

In [12]:
totaldata.head()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family,Title
0,1,3,male,22.0,7.2500,NaN,S,2,0,1,0,Mr
1,2,1,female,38.0,71.2833,C85,C,2,0,1,0,Mrs
2,3,3,female,26.0,7.9250,NaN,S,1,1,0,0,Miss
3,4,1,female,35.0,53.1000,C123,S,2,0,1,0,Mrs
4,5,3,male,35.0,8.0500,NaN,S,1,1,0,0,Mr


In [13]:
grouped = totaldata.groupby(['Sex','Pclass','Title'])

In [14]:
grouped.median()

PassengerId   Age       Fare  FamilySize  Alone  \
Sex    Pclass Title                                                           
female 1      Dona               1306.0  39.0  108.90000         1.0    1.0   
              Dr                  797.0  49.0   25.92920         1.0    1.0   
              Lady                557.0  48.0   39.60000         2.0    0.0   
              Miss                513.0  30.0  108.65415         1.0    1.0   
              Mlle                676.5  24.0   59.40210         1.0    1.0   
              Mme                 370.0  24.0   69.30000         1.0    1.0   
              Mrs                 857.0  45.0   78.26670         2.0    0.0   
              the Countess        760.0  33.0   86.50000         1.0    1.0   
       2      Miss                606.5  20.0   20.25000         1.5    0.5   
              Mrs                 547.0  30.5   26.00000         2.0    0.0   
              Ms                  444.0  28.0   13.00000         1.0    1.0   
       3      Miss                603.5  18.0    8.05000         1.0    1.0   
              Mrs                 658.0  31.0   15.50000         3.0    0.0   
              Ms                  980.0   NaN    7.75000         1.0    1.0   
male   1      Capt                746.0  70.0   71.00000         3.0    0.0   
              Col                 859.0  54.5   32.00000         1.0    1.0   
              Don                  31.0  40.0   27.72080         1.0    1.0   
              Dr                  661.0  47.0   81.85830         3.0    0.0   
              Jonkheer            823.0  38.0    0.00000         1.0    1.0   
              Major               493.5  48.5   28.52500         1.0    1.0   
              Master              803.0   6.0  134.50000         4.0    0.0   
              Mr                  634.0  41.5   47.10000         1.0    1.0   
              Sir                 600.0  49.0   56.92920         2.0    0.0   
       2      Dr                  358.5  38.5   12.25000         1.0    1.0   
              Master              550.0   2.0   26.00000         3.0    0.0   
              Mr                  723.5  30.0   13.00000         1.0    1.0   
              Rev                 738.0  41.5   13.00000         1.0    1.0   
       3      Master              789.0   6.0   22.35830         5.0    0.0   
              Mr                  640.5  26.0    7.89580         1.0    1.0   

                            Couple  Family  
Sex    Pclass Title                         
female 1      Dona             0.0     0.0  
              Dr               0.0     0.0  
              Lady             1.0     0.0  
              Miss             0.0     0.0  
              Mlle             0.0     0.0  
              Mme              0.0     0.0  
              Mrs              1.0     0.0  
              the Countess     0.0     0.0  
       2      Miss             0.0     0.0  
              Mrs              0.0     0.0  
              Ms               0.0     0.0  
       3      Miss             0.0     0.0  
              Mrs              0.0     1.0  
              Ms               0.0     0.0  
male   1      Capt             0.0     1.0  
              Col              0.0     0.0  
              Don              0.0     0.0  
              Dr               0.0     1.0  
              Jonkheer         0.0     0.0  
              Major            0.0     0.0  
              Master           0.0     1.0  
              Mr               0.0     0.0  
              Sir              1.0     0.0  
       2      Dr               0.0     0.0  
              Master           0.0     1.0  
              Mr               0.0     0.0  
              Rev              0.0     0.0  
       3      Master           0.0     1.0  
              Mr               0.0     0.0

In [15]:
totaldata["Age"] = totaldata.groupby(['Sex','Pclass','Title'])['Age'].transform(lambda x: x.fillna(x.median()))
totaldata["Fare"] = totaldata.groupby(['Sex','Pclass','Title'])['Fare'].transform(lambda x: x.fillna(x.median()))

In [16]:
totaldata.columns[totaldata.isnull().any()]

Index(['Age', 'Cabin', 'Embarked'], dtype='object')

In [17]:
totaldata['Embarked'].mode()

0    S
dtype: object

In [18]:
totaldata.Embarked.fillna('S',inplace=True)
totaldata.Cabin.fillna('U',inplace=True)
totaldata['Cabin'] = totaldata['Cabin'].map(lambda c : c[0])

In [19]:
totaldata.head()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family,Title
0,1,3,male,22.0,7.2500,U,S,2,0,1,0,Mr
1,2,1,female,38.0,71.2833,C,C,2,0,1,0,Mrs
2,3,3,female,26.0,7.9250,U,S,1,1,0,0,Miss
3,4,1,female,35.0,53.1000,C,S,2,0,1,0,Mrs
4,5,3,male,35.0,8.0500,U,S,1,1,0,0,Mr


In [20]:
# label encode categorical features in data given
cols = ['Sex','Cabin','Embarked','Title']

for x in cols:
    lbl = LabelEncoder()
    totaldata[x] = lbl.fit_transform(totaldata[x])

In [21]:
# split the merged data file into train and test respectively
data = totaldata.iloc[0:891]
test = totaldata.iloc[891:]

In [22]:
data.tail()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,FamilySize,Alone,Couple,Family,Title
886,887,2,1,27.0,13.00,8,2,1,1,0,0,15
887,888,1,0,19.0,30.00,1,2,1,1,0,0,9
888,889,3,0,18.0,23.45,8,2,4,0,0,1,9
889,890,1,1,26.0,30.00,2,0,1,1,0,0,12
890,891,3,1,32.0,7.75,8,1,1,1,0,0,12


In [23]:
# let's check cross validation score of the model
# cv score acts a unbiased estimate of models accuracy on unseen data

mod1 = GaussianNB()

In [24]:
# Naive Bayes
print(cross_val_score(mod1, data, target, cv=5, scoring=make_scorer(accuracy_score)))

[0.65921788 0.74860335 0.75280899 0.80898876 0.81355932]


In [25]:
test = test.fillna(method='ffill')

In [26]:
# make our first set of predictions
clf1 = GaussianNB()
clf1.fit(data, target)

GaussianNB(priors=None)

In [27]:
preds1 = clf1.predict(test)

In [28]:
sub1 = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':preds1})

In [29]:
## write submission files
sub1.to_csv('/home/priya/Downloads/TitanicSinking/Submissions/GenderSubmission.csv', index=False)